In [2]:
#https://medium.com/swlh/determining-a-cut-off-or-threshold-when-working-with-a-binary-dependent-target-variable-7c2342cf2a7c
!pip install --upgrade numpy 
!pip install plotly --upgrade
!pip install chart-studio --upgrade
!pip install botocore
!pip install ibm_boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement ibm_boto3 (from versions: none)
ERROR: No matching distribution found for ibm_boto3


In [4]:
!pip install ibm-cos-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 967.4/967.4 KB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 KB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.5 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.12.2-py3-none-any.whl size=73932 sha256=38fb64c751af917d65a77b4ad91254facd8ce31b05f35d06ac98467748584fc2
  Stored in directory: /root/.cache/pip/wheels/1e/16/92/84819122342c6fff13a3eb139780d6f738b51672bd2906243e
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.12.2-py3-none-any.whl size=564569 sha256=520cd4e006027a75a95269c0cc21909f13c5d98d1932c234a69070985d8b70ff
  Stored in dire

In [5]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy.dual as dual
import plotly as plotly
import pandas as pd
from botocore.client import Config
import ibm_boto3
import numpy as np
import numpy.dual as dual
from sklearn import metrics
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
def __iter__(self): return 0
#Un-Comment these options if you want to exapand the number of rows and columns of you see visually in the notebook.
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
!rm df_for_export-2.csv
!wget https://raw.githubusercontent.com/shadgriffin/machine_failure/master/df_for_export-2.csv
pd_data = pd.read_csv("df_for_export-2.csv", sep=",", header=0)

rm: cannot remove 'df_for_export-2.csv': No such file or directory
--2023-02-16 19:57:05--  https://raw.githubusercontent.com/shadgriffin/machine_failure/master/df_for_export-2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4768640 (4.5M) [text/plain]
Saving to: ‘df_for_export-2.csv’

df_for_export-2.csv 100%[===================>]   4.55M  --.-KB/s    in 0.08s   

2023-02-16 19:57:06 (60.4 MB/s) - ‘df_for_export-2.csv’ saved [4768640/4768640]



In [7]:
#2.0 Data Exporation
pd_data.head()

,ID,DATE,FAILURE_TARGET,P_FAIL
0,100003,4/5/16,0,0.177485
1,100003,3/30/16,0,0.208043
2,100003,3/31/16,0,0.194013
3,100003,4/1/16,0,0.158498
4,100003,4/2/16,0,0.190317


In [9]:
#3.0 Introducing a Gains Table
#3.1 Assign Deciles based on the probability
dfx=pd_data
#Sort the data by ID and Date
dfx=dfx.sort_values(by=["P_FAIL"], ascending=[False])
#add a very small random number to the probability to break ties
dfx['wookie'] = (np.random.randint(0, 100, dfx.shape[0]))/100000000000000000
dfx['P_FAIL']=dfx['P_FAIL']+dfx['wookie']
#Create deciles based on P_FAIL
dfx['DECILE'] = pd.qcut(dfx['P_FAIL'], 10, labels=np.arange(100, 0, -10))

In [10]:
dfx.head(3)

,ID,DATE,FAILURE_TARGET,P_FAIL,wookie,DECILE
134043,100483,11/9/16,1,0.975795,5.300000e-16,10
106945,100387,9/19/16,1,0.974342,3.700000e-16,10
106942,100387,9/16/16,1,0.974066,8.700000e-16,10


In [11]:
#Gains Table
# Find the minimum probability for each decile
tips_summedv = pd.DataFrame(dfx.groupby(['DECILE'])['P_FAIL'].min())
# Find the maximum probability of each decile
tips_summedw = pd.DataFrame(dfx.groupby(['DECILE'])['P_FAIL'].max())
# Find the Actual Failure rate for each decile.
tips_summedx = pd.DataFrame(dfx.groupby(['DECILE'])['FAILURE_TARGET'].mean())
#Sum the number of Failures in each decile.
tips_summedy = pd.DataFrame(dfx.groupby(['DECILE'])['FAILURE_TARGET'].sum())
# count the records in each decile
tips_summedz = pd.DataFrame(dfx.groupby(['DECILE'])['FAILURE_TARGET'].count())
#Aggregate the summaries into one dataframe
tips = pd.concat([tips_summedv,tips_summedw, tips_summedx, tips_summedy,tips_summedz], axis=1)
tips.columns = ['MIN_SCORE','MAX_SCORE','FAILURE_RATE','FAILURES', 'OBS']
tips=tips.sort_values(by=['DECILE'], ascending=[False])
gains=tips
#Find the number of cumulative failures by decile.
gains['CUML_FAILURES']=gains['FAILURES'].cumsum()
#Find the percentage of failures in each decile
gains['PCT_OF_FAILURES']=(gains.FAILURES)/(dfx['FAILURE_TARGET'].sum())*100
#Find the cumulative percentage of failures in each decile.
gains['CUML_PCT_OF_FAILURES']=gains.PCT_OF_FAILURES.cumsum()
#Format the final output
gains=gains[['OBS','MIN_SCORE','MAX_SCORE','FAILURES','FAILURE_RATE','PCT_OF_FAILURES','CUML_FAILURES','CUML_PCT_OF_FAILURES']]
gains

,OBS,MIN_SCORE,MAX_SCORE,FAILURES,FAILURE_RATE,PCT_OF_FAILURES,CUML_FAILURES,CUML_PCT_OF_FAILURES
DECILE,,,,,,,,
10,16521,0.634321,0.975795,3265,0.197627,69.959289,3265,69.959289
20,16520,0.455667,0.634312,818,0.049516,17.527319,4083,87.486608
30,16521,0.335289,0.455661,354,0.021427,7.585172,4437,95.071781
40,16520,0.252277,0.335284,143,0.008656,3.064067,4580,98.135847
50,16521,0.191109,0.252276,62,0.003753,1.328477,4642,99.464324
60,16520,0.146307,0.191104,20,0.001211,0.428541,4662,99.892865
70,16521,0.111849,0.146306,3,0.000182,0.064281,4665,99.957146
80,16520,0.080237,0.111848,2,0.000121,0.042854,4667,100.000000
90,16521,0.041927,0.080236,0,0.000000,0.000000,4667,100.000000


In [12]:
##4.1 Finding the best cut-off

In [13]:
dfx=pd_data
#Add a small negative random number to P_FAIL to break ties when grouping
dfx['wookie'] = (np.random.randint(0, 100, dfx.shape[0]))/100000000000000000
dfx['P_FAIL']=dfx['P_FAIL']+dfx['wookie']

In [14]:
dfx['GROUPS'] = pd.qcut(dfx['P_FAIL'], 10000, labels=False)
# find the minimum P_FAIL for each group.  This is a potential cut-off point.
tips_summedb = pd.DataFrame(dfx.groupby(['GROUPS'])['P_FAIL'].min())
#Find the number of Failures in each group
tips_summedz = pd.DataFrame(dfx.groupby(['GROUPS'])['FAILURE_TARGET'].sum())
#find the number of observations in each group
tips_summeda = pd.DataFrame(dfx.groupby(['GROUPS'])['FAILURE_TARGET'].count())
#append the summaries into one dataframe
tips = pd.concat([tips_summedb,tips_summedz, tips_summeda], axis=1)
tips.columns = ['CUT-OFF','FAILURES', 'OBS']
#find the number of non-failures
tips['NON_FAILURES']=tips.OBS-tips.FAILURES
#reset the index to make GROUPS a column
tips.reset_index(level=0, inplace=True)
#sort the dataframe by groups in descending order
tips=tips.sort_values(by=['GROUPS'], ascending=[False])
# Cumulative sum the failures, non-failures and observations
tips['INV_CUM_FAILURES'] = tips.FAILURES.cumsum()
tips['INV_CUM_NON_FAILURES'] = tips.NON_FAILURES.cumsum()
tips['TOTAL_OBS']=tips.OBS.sum()
#Sort the data by Groups ascending
tips=tips.sort_values(by=['GROUPS'], ascending=[True])
#calculate the total number of failures and non-failures
tips['CUM_FAILURES'] = tips.FAILURES.cumsum()
tips['CUM_NON_FAILURES'] = tips.NON_FAILURES.cumsum()
#find the total number of failures for the whole dataset.
tips['TOTAL_FAILURES']=tips.FAILURES.sum()
tips['TOTAL_NON_FAILURES']=tips.NON_FAILURES.sum()
#define the true positives for each cut-off
tips['TRUE_POSITIVES']=tips.INV_CUM_FAILURES
#define the false positives for each cut-off
tips['FALSE_POSITIVES']=tips.INV_CUM_NON_FAILURES
#define the true negatives for each cut-off
tips['TRUE_NEGATIVES']=tips.CUM_NON_FAILURES-tips.NON_FAILURES
#define the false negatives for each cut-off
tips['FALSE_NEGATIVES']=tips.CUM_FAILURES-tips.FAILURES
#double check the logic and arithmetic.
tips['OBS2']=tips.TRUE_POSITIVES+tips.FALSE_POSITIVES+tips.TRUE_NEGATIVES+tips.FALSE_NEGATIVES
# define the sensitvity for each cut-off
tips['SENSITIVITY']=tips['TRUE_POSITIVES']/(tips['TRUE_POSITIVES']+tips['FALSE_NEGATIVES'])
#define the specificity for each cut-off
tips['SPECIFICITY']=tips['TRUE_NEGATIVES']/(tips['FALSE_POSITIVES']+tips['TRUE_NEGATIVES'])
#define the false positive rate for each cut-off
tips['FALSE_POSITIVE_RATE']=1-tips['SPECIFICITY']
#define the false negative rate for each cut-off
tips['FALSE_NEGATIVE_RATE']=1-tips['SENSITIVITY']
tipsx=tips
gains=tipsx[['GROUPS','CUT-OFF','TRUE_POSITIVES','FALSE_POSITIVES','TRUE_NEGATIVES','FALSE_NEGATIVES','SENSITIVITY',
            'SPECIFICITY','FALSE_POSITIVE_RATE','FALSE_NEGATIVE_RATE']]
gains


,GROUPS,CUT-OFF,TRUE_POSITIVES,FALSE_POSITIVES,TRUE_NEGATIVES,FALSE_NEGATIVES,SENSITIVITY,SPECIFICITY,FALSE_POSITIVE_RATE,FALSE_NEGATIVE_RATE
0,0,0.001404,4667,160539,0,0,1.000000,0.000000,1.000000,0.000000
1,1,0.002718,4667,160522,17,0,1.000000,0.000106,0.999894,0.000000
2,2,0.003173,4667,160505,34,0,1.000000,0.000212,0.999788,0.000000
3,3,0.003342,4667,160489,50,0,1.000000,0.000311,0.999689,0.000000
4,4,0.003520,4667,160472,67,0,1.000000,0.000417,0.999583,0.000000
...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.958857,71,12,160527,4596,0.015213,0.999925,0.000075,0.984787
9996,9996,0.958927,54,12,160527,4613,0.011571,0.999925,0.000075,0.988429
9997,9997,0.961913,43,7,160532,4624,0.009214,0.999956,0.000044,0.990786
9998,9998,0.963536,32,1,160538,4635,0.006857,0.999994,0.000006,0.993143


In [15]:
#4.2 Finding the cut-off with the smallest misclassification rate, #assumption is that a false positive has the same value as a false negative. Often, this is not the case. 
tips=tipsx
#sum the false positives and false negatives.
tips['FALSE_CLASSIFICATIONS'] = tips.FALSE_POSITIVES+tips.FALSE_NEGATIVES
#estimate the false classification rate
tips['FALSE_CLASSIFICATION_RATE']=tips.FALSE_CLASSIFICATIONS/(tips.TOTAL_OBS)
gains=tips[['GROUPS','CUT-OFF','TRUE_POSITIVES','FALSE_POSITIVES','TRUE_NEGATIVES','FALSE_NEGATIVES','SENSITIVITY',
            'SPECIFICITY','FALSE_POSITIVE_RATE','FALSE_NEGATIVE_RATE','FALSE_CLASSIFICATIONS','FALSE_CLASSIFICATION_RATE']]
gains

,GROUPS,CUT-OFF,TRUE_POSITIVES,FALSE_POSITIVES,TRUE_NEGATIVES,FALSE_NEGATIVES,SENSITIVITY,SPECIFICITY,FALSE_POSITIVE_RATE,FALSE_NEGATIVE_RATE,FALSE_CLASSIFICATIONS,FALSE_CLASSIFICATION_RATE
0,0,0.001404,4667,160539,0,0,1.000000,0.000000,1.000000,0.000000,160539,0.971750
1,1,0.002718,4667,160522,17,0,1.000000,0.000106,0.999894,0.000000,160522,0.971648
2,2,0.003173,4667,160505,34,0,1.000000,0.000212,0.999788,0.000000,160505,0.971545
3,3,0.003342,4667,160489,50,0,1.000000,0.000311,0.999689,0.000000,160489,0.971448
4,4,0.003520,4667,160472,67,0,1.000000,0.000417,0.999583,0.000000,160472,0.971345
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.958857,71,12,160527,4596,0.015213,0.999925,0.000075,0.984787,4608,0.027892
9996,9996,0.958927,54,12,160527,4613,0.011571,0.999925,0.000075,0.988429,4625,0.027995
9997,9997,0.961913,43,7,160532,4624,0.009214,0.999956,0.000044,0.990786,4631,0.028032
9998,9998,0.963536,32,1,160538,4635,0.006857,0.999994,0.000006,0.993143,4636,0.028062


In [16]:
x1 = tips['CUT-OFF']
y1 = tips['FALSE_POSITIVE_RATE']
y2 = tips['FALSE_NEGATIVE_RATE']
y3 = tips['FALSE_CLASSIFICATION_RATE']
trace = go.Scatter(
    x = x1,
    y = y1,
    name='False Positive Rate')
trace2 = go.Scatter(
    x = x1,
    y = y2,
    name='False Negative Rate'
)
trace3 = go.Scatter(
    x = x1,
    y = y3,
    name='False Classification Rate'
)
layout = go.Layout(
    title='Mis-Classification Rates BY CUT OFF SCORE',
    xaxis=dict(
        title='CUT OFF SCORE',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='False Positive and False Negative Rates',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    showlegend=True,
)
    
data=[trace,trace2,trace3]  
fig = go.Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='styling-names')
plotly.offline.iplot(fig, filename='shapes-lines')

In [18]:
#4.3 Finding the cut-off when the costs of false positives and false negatives are not equal
tips=tipsx
#define the cost of a false positive and false negative
cost_of_a_false_positive=1
cost_of_a_false_negative=2
#convert the costs into weights that sum to one
fp_weight=(cost_of_a_false_positive)/(cost_of_a_false_positive+cost_of_a_false_negative)
fn_weight=(cost_of_a_false_negative)/(cost_of_a_false_positive+cost_of_a_false_negative)
#Create a weighted false classification rate based on the costs of a false positive and a false negative.
tips['FALSE_CLASSIFICATIONS_W'] = np.where(((2*(((fp_weight)*tips.FALSE_POSITIVES+(fn_weight)*tips.FALSE_NEGATIVES)) >= tips.TOTAL_OBS)), tips.TOTAL_OBS, 2*(((fp_weight)*tips.FALSE_POSITIVES+(fn_weight)*tips.FALSE_NEGATIVES)))
tips['FALSE_CLASSIFICATION_RATE_W']=tips.FALSE_CLASSIFICATIONS_W/(tips.TOTAL_OBS)
gains=tips[['GROUPS','CUT-OFF','TRUE_POSITIVES','FALSE_POSITIVES','TRUE_NEGATIVES','FALSE_NEGATIVES','SENSITIVITY',
            'SPECIFICITY','FALSE_POSITIVE_RATE','FALSE_NEGATIVE_RATE','FALSE_CLASSIFICATIONS_W','FALSE_CLASSIFICATION_RATE_W']]
gains

,GROUPS,CUT-OFF,TRUE_POSITIVES,FALSE_POSITIVES,TRUE_NEGATIVES,FALSE_NEGATIVES,SENSITIVITY,SPECIFICITY,FALSE_POSITIVE_RATE,FALSE_NEGATIVE_RATE,FALSE_CLASSIFICATIONS_W,FALSE_CLASSIFICATION_RATE_W
0,0,0.001404,4667,160539,0,0,1.000000,0.000000,1.000000,0.000000,107026.000000,0.647834
1,1,0.002718,4667,160522,17,0,1.000000,0.000106,0.999894,0.000000,107014.666667,0.647765
2,2,0.003173,4667,160505,34,0,1.000000,0.000212,0.999788,0.000000,107003.333333,0.647696
3,3,0.003342,4667,160489,50,0,1.000000,0.000311,0.999689,0.000000,106992.666667,0.647632
4,4,0.003520,4667,160472,67,0,1.000000,0.000417,0.999583,0.000000,106981.333333,0.647563
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.958857,71,12,160527,4596,0.015213,0.999925,0.000075,0.984787,6136.000000,0.037142
9996,9996,0.958927,54,12,160527,4613,0.011571,0.999925,0.000075,0.988429,6158.666667,0.037279
9997,9997,0.961913,43,7,160532,4624,0.009214,0.999956,0.000044,0.990786,6170.000000,0.037347
9998,9998,0.963536,32,1,160538,4635,0.006857,0.999994,0.000006,0.993143,6180.666667,0.037412


In [19]:
x1 = tips['CUT-OFF']
y1 = tips['FALSE_POSITIVE_RATE']
y2 = tips['FALSE_NEGATIVE_RATE']
y3 = tips['FALSE_CLASSIFICATION_RATE_W']
trace = go.Scatter(
    x = x1,
    y = y1,
    name='False Positive Rate')
trace2 = go.Scatter(
    x = x1,
    y = y2,
    name='False Negative Rate'
)
trace3 = go.Scatter(
    x = x1,
    y = y3,
    name='Weighted False Classification Rate'
)
layout = go.Layout(
    title='Weighted Mis-Classification Rates BY CUT OFF SCORE',
    xaxis=dict(
        title='CUT OFF SCORE',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Weighted False Positive and False Negative Rates',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    showlegend=True,
)
    
data=[trace,trace2,trace3]  
fig = go.Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='styling-names')
plotly.offline.iplot(fig, filename='shapes-lines')